 # Pipeline Resnet
Thanks to https://www.kaggle.com/dimitreoliveira/aptos-blindness-detection-eda-and-keras-resnet50

The novelty of this kernel is the attention mechanism that I tried to implement after the resnet50. I display the images with their masks generated by the attention network. This is a first attempt, and needs to be explored a bit more. For me it improved the predictions by quite a lot.   

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import shutil
from tqdm import tqdm

DATA_PATH = "../input/aptos2019-blindness-detection/"
TRAIN_PATH = DATA_PATH + 'train_images/'
TEST_PATH = DATA_PATH + 'test_images/'

print(os.listdir(DATA_PATH))

from glob import glob 


from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

# Load data

In [ ]:
train = pd.read_csv(DATA_PATH + "train.csv")
test = pd.read_csv(DATA_PATH + "test.csv")

train["id_code"] = train["id_code"].apply(lambda x: x + ".png")
test["id_code"] = test["id_code"].apply(lambda x: x + ".png")

train['diagnosis'] = train['diagnosis'].astype('str')

train.head(10)

**Preprocessing the data before training (and using flow_from_directory instead of flow_from_dataframe) cuts training time within kaggle kernel by factor >10 !**

In [ ]:
train_dest = 'base_dir/train'
test_dest =  'base_dir/test/'

if os.path.exists(train_dest):
    shutil.rmtree(train_dest)
if os.path.exists(test_dest):
    shutil.rmtree(test_dest)
    
# creating train and validation directories and subdirectories

for subf in ["0", "1","2","3","4"]:
    os.makedirs(os.path.join(train_dest, subf))

# Creating test directory
os.makedirs(test_dest + 'none/')
        

In [ ]:
def crop_image1(img,tol=7):
    # img is image data
    # tol  is tolerance
        
    mask = img>tol
    return img[np.ix_(mask.any(1),mask.any(0))]

# The above code work only for 1-channel. Here is my simple extension for 3-channels image
def crop_image(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        h,w,_=img.shape
#         print(h,w)
        img1=cv2.resize(crop_image1(img[:,:,0]),(w,h))
        img2=cv2.resize(crop_image1(img[:,:,1]),(w,h))
        img3=cv2.resize(crop_image1(img[:,:,2]),(w,h))
        
#         print(img1.shape,img2.shape,img3.shape)
        img[:,:,0]=img1
        img[:,:,1]=img2
        img[:,:,2]=img3
        return img

In [ ]:
from cv2 import imread, cvtColor, resize, addWeighted, GaussianBlur, COLOR_BGR2GRAY, imwrite

IMAGE_SIZE = 224


for label in ['0', '1', '2', '3', '4']:
    for image in tqdm(train[train['diagnosis'] == label].values):

        fname = image[0]

        src = os.path.join(TRAIN_PATH, fname) 
        dst = os.path.join(train_dest, label, fname)
        
        im = imread(src)
        im = cvtColor(im, COLOR_BGR2GRAY)
        
        im = crop_image1(im)
        
        resized_image = resize(im, (IMAGE_SIZE, IMAGE_SIZE))
        resized_image = addWeighted(resized_image, 4, GaussianBlur(resized_image, (0,0), IMAGE_SIZE/10), -4, 128)
        
        status = imwrite(dst, resized_image)
        
   
print('\nPreparing test images...')      
for image in tqdm(test.values):

        fname = image[0]

        src = os.path.join(TEST_PATH, fname) 
        dst = os.path.join(test_dest  + 'none/', fname)
        
        im = imread(src)
        im = cvtColor(im, COLOR_BGR2GRAY)
        resized_image = resize(im, (IMAGE_SIZE, IMAGE_SIZE))
        resized_image = addWeighted(resized_image , 4, GaussianBlur(resized_image, (0,0), IMAGE_SIZE/10), -4, 128)
        
        imwrite(dst, resized_image)

**Display some preprocessed images**

In [ ]:
from matplotlib.pyplot import figure, imshow
from cv2 import imread

SEED = 101

fig = figure(figsize=(25, 16))

for class_id in ['0', '1', '2', '3', '4']:
    for i, (idx, row) in enumerate(train.loc[train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, int(class_id) * 5 + i + 1, xticks=[], yticks=[])
        path= os.path.join(train_dest, class_id, row['id_code'])
        image = imread(path)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))

        imshow(image)
        ax.set_title('Label: %d-%d-%s' % (int(class_id), idx, row['id_code']) )

# Creating the data generators

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

batch_size_train = 32
batch_size_test = 1

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1,
                                   horizontal_flip=True, 
                                   vertical_flip=True,
                                   rotation_range=20,
                                   zoom_range= 0.2,
                                   featurewise_center=True,
                                   featurewise_std_normalization=True,
                                   zca_whitening=True,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,)

train_gen = train_datagen.flow_from_directory(directory=train_dest,
                                              batch_size= batch_size_train,
                                              class_mode= 'categorical',
                                              target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                              subset='training')

valid_gen = train_datagen.flow_from_directory(directory=train_dest,
                                              batch_size= batch_size_train,
                                              class_mode= 'categorical',
                                              target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                              subset='validation')

test_datagen = ImageDataGenerator(rescale=1./255)

test_gen = test_datagen.flow_from_directory(directory=test_dest,
                                            batch_size= batch_size_test,
                                            class_mode= None,
                                            target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                            shuffle=False)

# Define the model 

In [ ]:
from tensorflow import math, cast, float32
from tensorflow.nn import softmax
from tensorflow.keras.losses import CategoricalCrossentropy

def custom_loss(y_true, y_pred):
    """
    Define your code here. You can now use `weights` directly
    in this function
    """
    custom_cce = math.add(math.multiply(y_true, -1*math.log(0.001+ y_pred*0.999)), 
                          math.multiply(1-y_true, -1*math.log(1-y_pred*0.999)))
    custom_cce = math.reduce_sum(custom_cce, 1) 
    
    weights = cast(1 + math.square(math.abs(math.argmax(y_pred, axis=1)-math.argmax(y_true, axis=1)))/100, float32)
    
    
    cce = CategoricalCrossentropy()
   
    return math.multiply(cce(y_pred,y_true), weights)


In [ ]:
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras import Model, layers
from keras.models import load_model, model_from_json
from keras.layers import GlobalAveragePooling2D, Dropout, Dense, Input

print('Creating model... ', end=' ')

input_tensor = Input(shape=(IMAGE_SIZE,IMAGE_SIZE,3))
conv_base = ResNet50(include_top=False, weights=None, input_tensor=input_tensor)

conv_base.load_weights('../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')

# Attention network

#a_map = layers.Conv2D(1024, 1, strides=(1, 1), padding="same", activation='relu')(conv_base.output)

a_map = layers.Conv2D(516, 1, strides=(1, 1), padding="same", activation='relu')(conv_base.output)

#a_map = layers.Conv2D(64, 1, strides=(1, 1), padding="same", activation='relu')(a_map)

a_map = layers.Conv2D(1, 1, strides=(1, 1), padding="same", activation='relu')(a_map)

#a_map = layers.Conv2D(1024, 1, strides=(1, 1), padding="same", activation='relu')(a_map)

a_map = layers.Conv2D(2048, 1, strides=(1, 1), padding="same", activation='sigmoid')(a_map)



res = layers.Multiply()([conv_base.output, a_map])
    
x = GlobalAveragePooling2D()(res)
x = Dropout(0.5)(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(5, activation='softmax', name='final_output')(x)
model = Model(input_tensor, predictions)

model.summary()

print("Done !")

# Train

**Training upper layers**

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam

train_steps = train_gen.n//train_gen.batch_size
val_steps = valid_gen.n//valid_gen.batch_size

for layer in model.layers:
    layer.trainable = False

for i in range(-9, 0):
    model.layers[i].trainable = True
    
optimizer = Adam(lr=1e-4)
model.compile(optimizer=optimizer, loss='categorical_crossentropy',  metrics=['accuracy'])

history_warmup = model.fit_generator(train_gen,
                                     steps_per_epoch=train_steps,
                                     validation_data=valid_gen,
                                     validation_steps=val_steps,
                                     epochs=2).history

**Training all layers**

In [ ]:
for layer in model.layers:
    layer.trainable = True

earlystopper = EarlyStopping(monitor='val_loss', patience=7, verbose=1, restore_best_weights=True)
reducel = ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=1, factor=0.3, min_lr=1e-6)

optimizer = Adam(1e-4)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])



history = model.fit_generator(train_gen,
                              steps_per_epoch=train_steps,
                              validation_data=valid_gen,
                              validation_steps=val_steps,
                              epochs=25,
                              callbacks=[reducel, earlystopper]).history

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

training_data = {'loss': history_warmup['loss'] + history['loss'],
                 'val_loss': history_warmup['val_loss'] + history['val_loss'],
                 'acc': history_warmup['acc'] + history['acc'],
                 'val_acc': history_warmup['val_acc'] + history['val_acc']}

sns.set_style("whitegrid")
fig, (ax1, ax2) = plt.subplots(2, 1, sharex='col', figsize=(20, 14))

ax1.plot(history['loss'], label='Train loss')
ax1.plot(history['val_loss'], label='Validation loss')
ax1.legend(loc='best')
ax1.set_title('Loss')

ax2.plot(history['acc'], label='Train Accuracy')
ax2.plot(history['val_acc'], label='Validation accuracy')
ax2.legend(loc='best')
ax2.set_title('Accuracy')

plt.xlabel('Epochs')
sns.despine()
plt.show()

# Analysis of the attention mechanism

In [ ]:
from cv2 import resize

def show_image_mask(path):

    im = imread(path)

    exctraction_model = Model(input_tensor, res)
    im = np.array([im])
    w = exctraction_model.predict(im)
    im_res = np.squeeze(im)[:,:,0]
    
    # emphasizing the attention for comparaisong purpose
    im_w = np.power(resize(np.mean(w[0], axis=2), (224, 224)),4)
    imshow(np.multiply(im_w, im_res), cmap = 'gray')

In [ ]:
fig = figure(figsize=(25, 16))

for class_id in ['0', '1', '2', '3', '4']:
    for i, (idx, row) in enumerate(train.loc[train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, int(class_id) * 5 + i + 1, xticks=[], yticks=[])
        path= os.path.join(train_dest, class_id, row['id_code'])
        show_image_mask(path)
        ax.set_title('Label: %d-%d-%s' % (int(class_id), idx, row['id_code']) )

# Predictions

In [ ]:
final_valid_datagen = ImageDataGenerator(rescale=1./255)

final_valid_gen = train_datagen.flow_from_directory(directory= train_dest,
                                              batch_size= 1,
                                              class_mode= 'categorical',
                                              target_size= (IMAGE_SIZE, IMAGE_SIZE),
                                              shuffle=False)

STEP_SIZE_TEST = final_valid_gen.n//final_valid_gen.batch_size
print('Start predictions...', end=' ')
preds = model.predict_generator(final_valid_gen, steps=STEP_SIZE_TEST)
print('Done !')
predictions = [np.argmax(pred) for pred in preds]

gt= list(final_valid_gen.labels)

In [ ]:
from sklearn.metrics import confusion_matrix
from seaborn import heatmap

plt.figure(figsize=(10, 10))

cm = confusion_matrix(gt, predictions, labels=[0,1,2,3,4])
heatmap(cm, annot=True, cbar=False, cmap='Blues', fmt='g', vmax=300, vmin=0)
plt.show()

In [ ]:
test_gen.reset()
STEP_SIZE_TEST = test_gen.n//test_gen.batch_size
print("Predictions begins...")
preds = model.predict_generator(test_gen, steps=STEP_SIZE_TEST)
print("Predictions done !")
predictions = [np.argmax(pred) for pred in preds]

filenames = test_gen.filenames
results = pd.DataFrame({'id_code':filenames, 'diagnosis':predictions})
results['id_code'] = results['id_code'].map(lambda x: str(x)[:-4])
results['id_code'] = results['id_code'].apply(lambda x: x.split('/')[1])
results.to_csv('submission.csv',index=False)
results.head(10)

In [ ]:
df = pd.read_csv("submission.csv")
print(df["diagnosis"].value_counts())
print(preds)

In [ ]:
if os.path.exists('valid/'):
    shutil.rmtree('valid/')
if os.path.exists(train_dest):
    shutil.rmtree(train_dest)
if os.path.exists(test_dest):
    shutil.rmtree(test_dest)